In [1]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
import spacy
from spacy_langdetect import LanguageDetector
from spacy.language import Language
from googletrans import Translator
import time
translator = Translator()

In [2]:
nlp = spacy.load('en_core_web_trf')

In [3]:
def get_lang_detector(nlp, name):
    return LanguageDetector()
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)

In [4]:
nlp_it = spacy.load("it_core_news_lg")
nlp_en = spacy.load("en_core_web_trf")
nlp_ge = spacy.load("de_dep_news_trf")
nlp_fr = spacy.load("fr_dep_news_trf")
nlp_es = spacy.load("es_dep_news_trf")

In [5]:
q_txt = []
with open("/data/Sepehr-data/istella22/queries.test.jsonl", "r") as f:
    for line in f:
        q_txt.append(json.loads(line))

In [6]:
query_text_df = pd.DataFrame(q_txt)
query_text_np = query_text_df.values
query_text_emb = pd.DataFrame(columns=["query_id", "query_text", "query_emb", "query_lang"])

In [7]:
embeddings = np.zeros(300)
for i in tqdm(range(len(query_text_np))):
    en_embeddings = nlp_en(query_text_np[i][1]).vector
    it_embeddings = nlp_it(query_text_np[i][1]).vector
    ge_embeddings = nlp_ge(query_text_np[i][1]).vector
    fr_embeddings = nlp_fr(query_text_np[i][1]).vector
    es_embeddings = nlp_es(query_text_np[i][1]).vector
    
    emb_list = [en_embeddings, it_embeddings, ge_embeddings, fr_embeddings, es_embeddings]
    #count empty embeddings
    count = 0
    for j in range(len(emb_list)):
        if len(emb_list[j]) != 0:
            count += 1
            emb_index = j
    if count == 1:
        query_text_emb = pd.concat([query_text_emb, pd.DataFrame([[query_text_np[i][0], query_text_np[i][1], emb_list[emb_index], 'na']], columns=["query_id", "query_text", "query_emb", "query_lang"])], ignore_index=True)
    elif count > 1:
        print("more than one language detected")
        doc = nlp(query_text_np[i][1])
        detect_language = doc._.language['language']
        if detect_language == 'en':
            query_text_emb = pd.concat([query_text_emb, pd.DataFrame([[query_text_np[i][0], query_text_np[i][1], en_embeddings, 'en']], columns=["query_id", "query_text", "query_emb", "query_lang"])], ignore_index=True)
        elif detect_language == 'it':
            query_text_emb = pd.concat([query_text_emb, pd.DataFrame([[query_text_np[i][0], query_text_np[i][1], it_embeddings, 'it']], columns=["query_id", "query_text", "query_emb", "query_lang"])], ignore_index=True)
        elif detect_language == 'de':
            query_text_emb = pd.concat([query_text_emb, pd.DataFrame([[query_text_np[i][0], query_text_np[i][1], ge_embeddings, 'de']], columns=["query_id", "query_text", "query_emb", "query_lang"])], ignore_index=True)
        elif detect_language == 'fr':
            query_text_emb = pd.concat([query_text_emb, pd.DataFrame([[query_text_np[i][0], query_text_np[i][1], fr_embeddings, 'fr']], columns=["query_id", "query_text", "query_emb", "query_lang"])], ignore_index=True)
        elif detect_language == 'es':
            query_text_emb = pd.concat([query_text_emb, pd.DataFrame([[query_text_np[i][0], query_text_np[i][1], es_embeddings, 'es']], columns=["query_id", "query_text", "query_emb", "query_lang"])], ignore_index=True)
        else:
            query_text_emb = pd.concat([query_text_emb, pd.DataFrame([[query_text_np[i][0], query_text_np[i][1], np.zeros(300), 'other']], columns=["query_id", "query_text", "query_emb", "query_lang"])], ignore_index=True)
    

100%|██████████| 2198/2198 [05:17<00:00,  6.92it/s]


In [8]:
query_text_emb.to_pickle("query_text_emb.pkl")